In [193]:
import os
import pandas as pd
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.spatial.distance import cityblock as manhattan_distance
from scipy.spatial.distance import minkowski as minkowski_distance
from scipy.stats import spearmanr
import tensorflow_hub as hub
from transformers import AutoTokenizer, AutoModel

In [194]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [195]:
dir_path = "C:\\Users\\Hassa\\OneDrive\\Desktop\\PRODUCT_SCOUT\\Back-End\\processed_data.csv"
df = pd.read_csv(dir_path)

temp = []
cats = []
for i in range(len(df)):
    temp.append(df["Category"][i])
    temp.append(df["Sub-Category"][i])
    temp.append(df["Sub-Sub-Category"][i])
    cats.append(temp)
    temp = []

categories = []
for item in cats:
    if item not in categories:
        categories.append(item)

In [196]:
category_query_list = [category[0] + " " + category[1] + " " + category[2] for category in categories]
category_query_embeddings_categorical = model.encode(category_query_list)
category_query_list_hierarchical = [category[0] for category in categories] + [category[1] for category in categories] + [category[2] for category in categories]
category_query_embeddings_hierarchical = model.encode(category_query_list_hierarchical)

In [197]:
user_query = "men shirts"

In [198]:
category_scores = []
user_query_embedding = model.encode([user_query])[0]
for i, category in enumerate(categories):
    score = cosine_similarity(user_query_embedding.reshape(1, -1), category_query_embeddings_categorical[i].reshape(1, -1))
    category_scores.append((category, score))
sorted_scores_categorical = sorted(category_scores, key=lambda x: x[1], reverse=True)

category_scores_hierarchical = []
user_query_embedding = model.encode([user_query])[0]
for i, category in enumerate(categories):
    category1_score = cosine_similarity(user_query_embedding.reshape(1, -1), category_query_embeddings_hierarchical[i].reshape(1, -1))
    category2_score = cosine_similarity(user_query_embedding.reshape(1, -1), category_query_embeddings_hierarchical[i+len(categories)].reshape(1, -1))
    category3_score = cosine_similarity(user_query_embedding.reshape(1, -1), category_query_embeddings_hierarchical[i+len(categories)*2].reshape(1, -1))
    score = (category1_score + category2_score + category3_score) / 3
    category_scores_hierarchical.append((category, score))

sorted_scores_hierarchical = sorted(category_scores_hierarchical, key=lambda x: x[1], reverse=True)

SCORES1 = []
SCORES1.append(sorted_scores_categorical[0][0])
SCORES1.append(sorted_scores_categorical[1][0])
SCORES1.append(sorted_scores_categorical[2][0])

SCORES2 = []
SCORES2.append(sorted_scores_hierarchical[0][0])
SCORES2.append(sorted_scores_hierarchical[1][0])
SCORES2.append(sorted_scores_hierarchical[2][0])

combined_list = SCORES1 + SCORES2
counts = {}
for lst in combined_list:
    key = tuple(lst)
    if key in counts:
        counts[key] += 1
    else:
        counts[key] = 1

sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
for key, count in sorted_counts:
    if list(key) in SCORES1 or list(key) in SCORES2:
        categoric_mapping = list(key)
        break
        
print(categoric_mapping)

['Fashion', 'Men', 'Clothing']


In [199]:
output_directory = "C:\\Users\\Hassa\\OneDrive\\Desktop\\PRODUCT_SCOUT\\Back-End"

with open(os.path.join(output_directory, 'categories.pkl'), 'wb') as f:
    pickle.dump(categories, f)

with open(os.path.join(output_directory, 'category_embeddings_categorical.pkl'), 'wb') as f:
    pickle.dump(category_query_embeddings_categorical, f)
    
with open(os.path.join(output_directory, 'category_embeddings_hierarchical.pkl'), 'wb') as f:
    pickle.dump(category_query_embeddings_hierarchical, f)